<a href="https://colab.research.google.com/github/sanyamChaudhary27/ML_models_from_scratch/blob/main/DecisionTreeRegressor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
from dataclasses import dataclass

In [23]:
@dataclass(init=False)
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value
    def is_leaf_node(self):
        return self.value is not None

In [24]:
class DecisionTreeRegressor:
    def __init__(self, max_depth=100, min_samples_split=2, n_features=None):
        self.n_features = n_features
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.root = None
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_feats = X.shape
        if (depth>=self.max_depth) or (n_samples<self.min_samples_split) or (np.var(y)==0):
            leaf_value = np.mean(y)
            return Node(value=leaf_value)
        feat_idx = np.random.choice(n_feats, self.n_features, replace=True)

        best_feat, best_threshold = self._best_split(X, y, feat_idx)
        if best_feat is None:
            leaf_value = np.mean(y)
            return Node(value = leaf_value)
        left_idx, right_idx = self._split(X[:,best_feat], best_threshold)
        left = self._grow_tree(X[left_idx,:], y[left_idx], depth+1)
        right = self._grow_tree(X[right_idx,:], y[right_idx], depth+1)
        return Node(best_feat, best_threshold, left, right)
    def _best_split(self, X, y, feat_idx):
        best_reduction = -1
        split_idx, split_thresh = None, None
        parent_var = np.var(y)
        for feat in feat_idx:
            X_column = X[:, feat]
            thresholds = np.unique(X_column)
            for thr in thresholds:
                reduction = self._variance_reduction(y, X_column, thr, parent_var)
                if reduction > best_reduction:
                    best_reduction = reduction
                    split_idx = feat
                    split_thresh = thr
        return split_idx, split_thresh
    def _variance_reduction(self, y, X_column, threshold, parent_var):
        left_idxs, right_idxs = self._split(X_column, threshold)
        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        var_l, var_r = np.var(y[left_idxs]), np.var(y[right_idxs])
        weighted_child_var = ((n_l / n) * var_l) + ((n_r / n) * var_r)
        return parent_var - weighted_child_var

    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten()
        right_idxs = np.argwhere(X_column > split_thresh).flatten()
        return left_idxs, right_idxs

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [33]:
from sklearn.datasets import make_regression

# Generate a synthetic dataset for regression
X, y = make_regression(n_samples=10000, n_features=5, n_informative=3, noise=0.4, random_state=42)

print("Shape of features (X):", X.shape)
print("Shape of target (y):", y.shape)

Shape of features (X): (10000, 5)
Shape of target (y): (10000,)


In [50]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [69]:
reg = DecisionTreeRegressor(min_samples_split=2, max_depth=16)
reg.fit(X, y)

In [70]:
predictions = reg.predict(X_test)

In [63]:
from sklearn.metrics import mean_squared_error

In [64]:
def rmse(y, y_pred):
    return float(np.sqrt(mean_squared_error(y, y_pred)))

In [71]:
Error = rmse(y_test, predictions)
Error

0.22224213875866425

# Decision Tree Regressor from Scratch

This notebook implements a custom Decision Tree Regressor from scratch and demonstrates its usage on a synthetically generated dataset.

## 1. Importing Libraries
We start by importing necessary libraries such as `numpy` for numerical operations, `pandas` (though not directly used in the final model, often useful), `dataclasses` for defining data structures, and `sklearn.datasets` and `sklearn.model_selection` for data generation and splitting.

## 2. Node Class Definition
The `Node` class serves as the building block for our decision tree. It stores information about a split (feature, threshold, left/right children) or a leaf node's value. It also includes a helper method `is_leaf_node()`.

## 3. DecisionTreeRegressor Class Definition
This is the core of our custom implementation. Key methods include:
- `__init__`: Initializes the regressor with parameters like `max_depth`, `min_samples_split`, and `n_features`.
- `fit`: Starts the tree growing process by calling `_grow_tree`.
- `_grow_tree`: Recursively builds the tree, handling stopping conditions (max depth, min samples, pure node).
- `_best_split`: Finds the best feature and threshold to split a node based on variance reduction.
- `_variance_reduction`: Calculates the reduction in variance achieved by a split.
- `_split`: Divides the data into left and right subsets based on a threshold.
- `predict`: Predicts target values for new data by traversing the tree.
- `_traverse_tree`: Helper method for tree traversal during prediction.

## 4. Synthetic Data Generation
A synthetic regression dataset is generated using `sklearn.datasets.make_regression` with 100 samples and 5 features. This provides a suitable dataset to test our custom regressor.

## 5. Model Training and Prediction
The `DecisionTreeRegressor` is initialized with `min_samples_split=2` and `max_depth=3` and then trained (`fit`) on the generated `X` and `y` data. Predictions are then made on the same `X` data (for demonstration purposes).

## 6. Model Evaluation
The actual and predicted values are displayed to give an initial visual sense of the model's performance. For a more rigorous evaluation, metrics like Mean Squared Error (MSE) or Root Mean Squared Error (RMSE) could be calculated using `sklearn.metrics`.